In [241]:
# get WRDS data
import wrds
conn = wrds.Connection(wrds_username='deadtomb')

Loading library list...
Done


In [242]:
import pandas as pd

In [243]:
import numpy as np

In [20]:
conn.list_tables(library='crsp_a_ccm')

['ccm_lookup',
 'ccm_qvards',
 'ccmxpf_linktable',
 'ccmxpf_lnkhist',
 'ccmxpf_lnkrng',
 'ccmxpf_lnkused',
 'comphead',
 'comphist',
 'compmaster',
 'cst_hist',
 'sechead',
 'sechist']

In [21]:
conn.describe_table(library='crsp_a_ccm', table='ccmxpf_linktable')

Approximately 85574 rows in crsp_a_ccm.ccmxpf_linktable.


,name,nullable,type,comment
0,gvkey,True,VARCHAR(6),Standard and Poor's Identifier
1,linkprim,True,VARCHAR(1),Primary Link Marker
2,liid,True,VARCHAR(3),Security-level Identifer
3,linktype,True,VARCHAR(2),Link Type Code
4,lpermno,True,DOUBLE PRECISION,Historical CRSP PERMNO Link to COMPUSTAT Record
5,lpermco,True,DOUBLE PRECISION,Historical CRSP PERMCO Link to COMPUSTAT Record
6,usedflag,True,DOUBLE PRECISION,Flag marking whether link is used in building ...
7,linkdt,True,DATE,First Effective Date of Link
8,linkenddt,True,DATE,Last Effective Date of Link


In [22]:
link_table = conn.raw_sql("""
    SELECT gvkey, lpermno AS permno, linktype, linkprim, linkdt, linkenddt
    FROM crsp.ccmxpf_linktable
    WHERE linktype IN ('LU', 'LC') AND linkprim IN ('P', 'C')
""")

In [23]:
display(link_table.head())

,gvkey,permno,linktype,linkprim,linkdt,linkenddt
0,001000,25881.0,LU,P,1970-11-13,1978-06-30
1,001001,10015.0,LU,P,1983-09-20,1986-07-31
2,001002,10023.0,LC,C,1972-12-14,1973-06-05
3,001003,10031.0,LU,C,1983-12-07,1989-08-16
4,001004,54594.0,LU,P,1972-04-24,None


In [245]:
compustat_data = conn.raw_sql("""
    SELECT gvkey, fyear, datadate, at, ceq, csho, prcc_f, dlc, dltt, dvt, dvc, oibdp, sale, ppent, xad, xrd, fyr, exchg
    FROM comp.funda
    WHERE indfmt = 'INDL' AND datafmt = 'STD' AND popsrc = 'D' AND consol = 'C' AND fyear BETWEEN 1950 AND 2021
""")

In [246]:
crsp_data = conn.raw_sql("""
    SELECT permno, EXTRACT(YEAR FROM date) AS fyear, hexcd
    FROM crsp.msf
    WHERE date BETWEEN '1950-01-01' AND '2021-12-31'
    GROUP BY permno, EXTRACT(YEAR FROM date), hexcd
""")

In [247]:
merged_data = pd.merge(compustat_data, link_table, on='gvkey', how='left')
merged_data = merged_data[(merged_data['linkdt'] <= merged_data['datadate']) & 
                          ((merged_data['linkenddt'] >= merged_data['datadate']) | merged_data['linkenddt'].isnull())]
merged_data = pd.merge(merged_data, crsp_data, on=['permno', 'fyear'], how='left')
merged_data = merged_data.drop_duplicates()
merged_data['xrd'] = merged_data['xrd'].fillna(0)

In [248]:
merged_data['mve'] = merged_data['csho'] * merged_data['prcc_f']
merged_data['market_value_of_assets'] = merged_data['at'] - merged_data['ceq'] + merged_data['mve']
merged_data['market_debt_ratio'] = (merged_data['dlc'] + merged_data['dltt']) / (merged_data['at'] - merged_data['ceq'] + merged_data['mve'])
merged_data['ln_market_value_of_assets'] = np.log(merged_data['market_value_of_assets'])
merged_data['profits_to_sales'] = merged_data['oibdp'] / merged_data['sale']
merged_data['tangible_assets'] = merged_data['ppent'] / merged_data['at']
merged_data['market_to_book_assets'] = merged_data['market_value_of_assets'] / merged_data['at']
merged_data['advertising_to_sales'] = merged_data['xad'] / merged_data['sale']
merged_data['rd_to_sales'] = merged_data['xrd'] / merged_data['sale']
merged_data['rd_positive'] = (merged_data['xrd'] > 0).astype(int)
merged_data['firm_age'] = merged_data['fyear'] - pd.to_datetime(merged_data['linkdt']).dt.year
merged_data['ln_firm_age'] = np.log(1 + merged_data['firm_age'])
merged_data = merged_data.sort_values(by=['gvkey', 'fyear'])
merged_data['prev_year_dvt'] = merged_data.groupby('gvkey')['dvt'].shift(1)
merged_data['prev_year_dvc'] = merged_data.groupby('gvkey')['dvc'].shift(1)


/Users/friend/anaconda3/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [249]:
independent_vars = ['ln_market_value_of_assets', 'ln_firm_age', 'profits_to_sales', 'tangible_assets', 'market_to_book_assets', 'advertising_to_sales', 'rd_to_sales', 'rd_positive']
for var in independent_vars:
    merged_data[var] = merged_data.groupby('gvkey')[var].shift(1)

In [250]:
merged_data = merged_data[merged_data['prev_year_dvc'] > 0]
merged_data = merged_data[merged_data['exchg'] == 11]
# merged_data = merged_data[merged_data['hexcd'] == 1]
merged_data = merged_data[(merged_data['fyear'] >= 1985) & (merged_data['fyear'] <= 2020)]
bounds = {}
variables = ['market_debt_ratio', 'ln_firm_age', 'ln_market_value_of_assets', 'market_to_book_assets', 'profits_to_sales', 'tangible_assets', 'advertising_to_sales', 'rd_to_sales']
for var in variables:
    bounds[var] = merged_data[var].quantile([0.01, 0.99])

for var in variables:
    lower_bound, upper_bound = bounds[var]
    merged_data = merged_data[(merged_data[var] >= lower_bound) & (merged_data[var] <= upper_bound)]


merged_data.to_csv('PartA_data.csv', index=False)

In [14]:
conn.close()

In [ ]:
# WRDS web query form proceeds to download the data in a CSV file
# data = pd.read_csv('CCM_output_Feb14.csv')

/var/folders/53/z0kd2hl56_1g6ntqlk8pfcgm0000gn/T/ipykernel_84803/999795585.py:2: DtypeWarning: Columns (2,15,17,33,37,40,41,42,952,953,955,956,957,962,976,977,982,987) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('CCM_output_Feb14.csv')


In [ ]:
data = pd.read_csv('PartA_data.csv')

In [251]:
data = merged_data

In [252]:
variables = ['market_debt_ratio', 'ln_market_value_of_assets', 'ln_firm_age', 'profits_to_sales', 'tangible_assets', 'market_to_book_assets', 'advertising_to_sales', 'rd_to_sales', 'rd_positive']
summary_stats = data[variables].describe().transpose()
summary_stats['count'] = summary_stats['count'].astype(int)
summary_stats = summary_stats[['count', 'mean', 'std', 'min', '50%', 'max']]
summary_stats.columns = ['Number of Observations', 'Mean', 'Standard Deviation', 'Minimum', 'Median', 'Maximum']


In [253]:
print(summary_stats)

                           Number of Observations      Mean  \
market_debt_ratio                           10094  0.176866   
ln_market_value_of_assets                   10094  8.510383   
ln_firm_age                                 10094  2.835833   
profits_to_sales                            10094  0.204212   
tangible_assets                             10094  0.254928   
market_to_book_assets                       10094  1.762658   
advertising_to_sales                        10094  0.027397   
rd_to_sales                                 10094  0.011700   
rd_positive                                 10094  0.381117   

                           Standard Deviation   Minimum    Median    Maximum  
market_debt_ratio                    0.142859  0.000000  0.146809   0.790811  
ln_market_value_of_assets            1.881171  4.110472  8.461485  13.179756  
ln_firm_age                          0.903136  0.000000  3.091042   4.007333  
profits_to_sales                     0.130631 -0.0453

In [ ]:
summary_stats.to_csv('PartA_summary_stats.csv')

In [254]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [255]:
# data['original_fyear'] = data['fyear']
# data = pd.get_dummies(data, columns=['fyear'], drop_first=True)
# data['fyear'] = data['original_fyear']
formula_fm = 'market_debt_ratio ~ ln_market_value_of_assets + ln_firm_age + profits_to_sales + tangible_assets + market_to_book_assets + advertising_to_sales + rd_to_sales + rd_positive'
# year_dummies = ' + '.join([f'fyear_{year}' for year in range(1986, 2020)])
# formula_with_year_dummies = formula_fm + ' + ' + year_dummies
formula_with_year_dummies = formula_fm + ' + C(fyear)'

In [13]:
model_white = smf.ols(formula=formula_with_year_dummies, data=data).fit(cov_type='HC0')
print(model_white.summary())
model_firm = smf.ols(formula=formula_with_year_dummies, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['gvkey']})
print(model_firm.summary())
model_year = smf.ols(formula=formula_with_year_dummies, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['fyr']})
print(model_year.summary())
model_firm_year = smf.ols(formula=formula_with_year_dummies, data=data).fit(cov_type='cluster', cov_kwds={'groups': data[['gvkey', 'fyr']].apply(tuple, axis=1)})
print(model_firm_year.summary())


                            OLS Regression Results                            
Dep. Variable:      market_debt_ratio   R-squared:                       0.203
Model:                            OLS   Adj. R-squared:                  0.199
Method:                 Least Squares   F-statistic:                     62.96
Date:                Sun, 23 Feb 2025   Prob (F-statistic):               0.00
Time:                        14:03:37   Log-Likelihood:                 6462.2
No. Observations:               10093   AIC:                        -1.284e+04
Df Residuals:                   10050   BIC:                        -1.253e+04
Df Model:                          42                                         
Covariance Type:                  HC0                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

/Users/friend/anaconda3/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 42, but rank is 11
  warnings.warn('covariance of constraints does not have full '


In [14]:
from linearmodels.panel import PanelOLS
from linearmodels.panel import FamaMacBeth

data.set_index(['gvkey', 'fyear'], inplace=True)
model_fm = FamaMacBeth.from_formula(formula_fm, data=data)
results_fm = model_fm.fit()
print(results_fm)


                             FamaMacBeth Estimation Summary                            
Dep. Variable:             market_debt_ratio   R-squared:                        0.6370
Estimator:                       FamaMacBeth   R-squared (Between):              0.6593
No. Observations:                      10093   R-squared (Within):               0.0341
Date:                       Sun, Feb 23 2025   R-squared (Overall):              0.6370
Time:                               14:04:26   Log-likelihood                    5743.2
Cov. Estimator:    Fama-MacBeth Standard Cov                                           
                                               F-statistic:                      2212.6
Entities:                               1112   P-value                           0.0000
Avg Obs:                              9.0764   Distribution:                 F(8,10085)
Min Obs:                              1.0000                                           
Max Obs:                        

In [260]:
def format_coefficients(model, coef_name, is_fm=False):
    coef = model.params[coef_name]
    se = model.std_errors[coef_name] if is_fm else model.bse[coef_name]
    pval = model.pvalues[coef_name]
    stars = ''
    if pval < 0.01:
        stars = '**'
    elif pval < 0.05:
        stars = '*'
    return f'{coef:.4f}{stars}\n({se:.4f})'

variables = ['ln_market_value_of_assets', 'ln_firm_age', 'market_to_book_assets', 'profits_to_sales', 'tangible_assets', 'advertising_to_sales', 'rd_to_sales', 'rd_positive']
results_table = pd.DataFrame(index=variables, columns=['I', 'II', 'III', 'IV', 'V'])
for var in variables:
    results_table.loc[var, 'I'] = format_coefficients(model_white, var)
    results_table.loc[var, 'II'] = format_coefficients(model_firm, var)
    results_table.loc[var, 'III'] = format_coefficients(model_year, var)
    results_table.loc[var, 'IV'] = format_coefficients(model_firm_year, var)
    results_table.loc[var, 'V'] = format_coefficients(results_fm, var, is_fm=True)
results_table.loc['R-squared'] = [
    f'{model_white.rsquared:.4f}',
    f'{model_firm.rsquared:.4f}',
    f'{model_year.rsquared:.4f}',
    f'{model_firm_year.rsquared:.4f}',
    f'{results_fm.rsquared:.4f}'
]
results_table.loc['Coefficient estimates'] = ['OLS', 'OLS', 'OLS', 'OLS', 'FM']
results_table.loc['Standard errors'] = ['White', 'CL - F', 'CL - T', 'CL - F&T', 'FM']
print(results_table)
results_table.to_csv('table2.csv')

                                             I                   II  \
ln_market_value_of_assets     0.0004\n(0.0004)     0.0004\n(0.0014)   
ln_firm_age                   0.0006\n(0.0008)     0.0006\n(0.0022)   
market_to_book_assets      -0.0776**\n(0.0009)  -0.0776**\n(0.0024)   
profits_to_sales            0.1725**\n(0.0083)   0.1725**\n(0.0253)   
tangible_assets             0.1616**\n(0.0033)   0.1616**\n(0.0109)   
advertising_to_sales         -0.0560\n(0.0315)    -0.0560\n(0.0879)   
rd_to_sales                -0.1566**\n(0.0380)    -0.1566\n(0.0938)   
rd_positive                -0.0172**\n(0.0018)  -0.0172**\n(0.0051)   
R-squared                               0.3090               0.3090   
Coefficient estimates                      OLS                  OLS   
Standard errors                          White               CL - F   

                                           III                   IV  \
ln_market_value_of_assets     0.0004\n(0.0020)     0.0004\n(0.0014)   
ln_fi

In [261]:
# Part C
formula = formula_fm
data.reset_index(inplace=True)
model_ols = smf.ols(formula=formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['gvkey']})

data['adj_y'] = data['market_debt_ratio'] - data.groupby('gvkey')['market_debt_ratio'].transform('mean')
model_adj_y = smf.ols(formula='adj_y ~ ' + ' + '.join([f'{var}' for var in variables]), data=data).fit(cov_type='cluster', cov_kwds={'groups': data['gvkey']})

# for var in variables:
#     data[f'adj_{var}'] = data[var] - data.groupby('gvkey')[var].transform('mean')
data['group_mean_dependent'] = data.groupby('gvkey')['market_debt_ratio'].transform('mean')
formula_avg_e = formula + ' + group_mean_dependent'
model_avg_e = smf.ols(formula=formula_avg_e, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['gvkey']})

data = data.set_index(['gvkey', 'fyear'])
model_gfe = PanelOLS.from_formula(formula + ' + EntityEffects', data=data).fit(cov_type='clustered', cluster_entity=True)


In [23]:
print(model_ols.summary())
print(model_adj_y.summary())
print(model_avg_e.summary())
print(model_gfe.summary)

                            OLS Regression Results                            
Dep. Variable:      market_debt_ratio   R-squared:                       0.183
Model:                            OLS   Adj. R-squared:                  0.182
Method:                 Least Squares   F-statistic:                     52.45
Date:                Sun, 23 Feb 2025   Prob (F-statistic):           3.02e-72
Time:                        14:40:09   Log-Likelihood:                 6336.6
No. Observations:               10093   AIC:                        -1.266e+04
Df Residuals:                   10084   BIC:                        -1.259e+04
Df Model:                           8                                         
Covariance Type:              cluster                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

In [263]:
def format_coefficients(model, coef_name, is_panel=False):
    coef = model.params[coef_name]
    se = model.std_errors[coef_name] if is_panel else model.bse[coef_name]
    pval = model.pvalues[coef_name]
    stars = ''
    if pval < 0.01:
        stars = '***'
    elif pval < 0.05:
        stars = '**'
    elif pval < 0.1:
        stars = '*'
    return f'{coef:.4f}{stars}\n({se:.4f})'

results_table = pd.DataFrame(index=variables, columns=['OLS', 'AdjY', 'AvgE', 'GFE'])

# Fill the DataFrame with formatted coefficients and standard errors
for var in variables:
    results_table.loc[var, 'OLS'] = format_coefficients(model_ols, f'{var}')
    results_table.loc[var, 'AdjY'] = format_coefficients(model_adj_y, f'{var}')
    results_table.loc[var, 'AvgE'] = format_coefficients(model_avg_e, f'{var}')
    results_table.loc[var, 'GFE'] = format_coefficients(model_gfe, f'{var}', is_panel=True)

# Add number of observations and R-squared at the bottom
results_table.loc['Observations'] = [
    f'{int(model_ols.nobs)}',
    f'{int(model_adj_y.nobs)}',
    f'{int(model_avg_e.nobs)}',
    f'{int(model_gfe.nobs)}'
]
results_table.loc['R-squared'] = [
    f'{model_ols.rsquared:.4f}',
    f'{model_adj_y.rsquared:.4f}',
    f'{model_avg_e.rsquared:.4f}',
    f'{model_gfe.rsquared:.4f}'
]

print(results_table)

                                            OLS                  AdjY  \
ln_market_value_of_assets   0.0128***\n(0.0024)   0.0013***\n(0.0005)   
ln_firm_age                -0.0230***\n(0.0035)      0.0003\n(0.0010)   
market_to_book_assets      -0.0547***\n(0.0033)  -0.0117***\n(0.0012)   
profits_to_sales           -0.1050***\n(0.0394)   -0.0207**\n(0.0083)   
tangible_assets             0.0764***\n(0.0184)     -0.0041\n(0.0034)   
advertising_to_sales          -0.1296\n(0.0908)   0.0728***\n(0.0220)   
rd_to_sales                -0.3548***\n(0.1255)      0.0237\n(0.0347)   
rd_positive                    0.0053\n(0.0083)      0.0026\n(0.0019)   
Observations                              10094                 10094   
R-squared                                0.1827                0.0224   

                                           AvgE                   GFE  
ln_market_value_of_assets   0.0017***\n(0.0005)   0.0134***\n(0.0041)  
ln_firm_age                   -0.0006\n(0.0010)     

In [ ]:
# Part E. replication of Table 7 of Petersen (2009) 

# Get the link table between CRSP and Compustat
link_table = conn.raw_sql("""
    SELECT gvkey, lpermno AS permno, linktype, linkprim, linkdt, linkenddt
    FROM crsp.ccmxpf_linktable
    WHERE linktype IN ('LU', 'LC') AND linkprim IN ('P', 'C')
""")

# Get the Compustat data
compustat_data = conn.raw_sql("""
    SELECT gvkey, fyear, datadate, at, ceq, csho, prcc_f, dlc, dltt, dvt, dvc, oibdp, sale, ppent, xad, xrd, fyr, exchg
    FROM comp.funda
    WHERE indfmt = 'INDL' AND datafmt = 'STD' AND popsrc = 'D' AND consol = 'C' AND fyear BETWEEN 1965 AND 2003
""")


In [120]:
conn.describe_table(library='crsp', table='stocknames')

Approximately 80451 rows in crsp.stocknames.


,name,nullable,type,comment
0,permno,True,INTEGER,PERMNO
1,namedt,True,DATE,None
2,nameenddt,True,DATE,None
3,shrcd,True,SMALLINT,Share Code as of Name Start Date
4,exchcd,True,SMALLINT,Exchange Code
5,siccd,True,INTEGER,SIC Code as of Name Start Date
6,ncusip,True,VARCHAR(8),CUSIP
7,ticker,True,VARCHAR(8),Ticker Symbol
8,comnam,True,VARCHAR(35),Company Name
9,shrcls,True,VARCHAR(4),Share Class as of Name Start Date


In [121]:
conn.describe_table(library='crsp', table='msf')

Approximately 5137059 rows in crsp.msf.


,name,nullable,type,comment
0,cusip,True,VARCHAR(8),CUSIP Header
1,permno,True,INTEGER,PERMNO
2,permco,True,INTEGER,PERMCO
3,issuno,True,INTEGER,Nasdaq Issue Number
4,hexcd,True,SMALLINT,Exchange Code Header
5,hsiccd,True,INTEGER,Standard Industrial Classification Code Header
6,date,True,DATE,Date of Observation
7,bidlo,True,"NUMERIC(11, 5)",Bid or Low Price
8,askhi,True,"NUMERIC(11, 5)",Ask or High Price
9,prc,True,"NUMERIC(11, 5)",Price or Bid/Ask Average


In [122]:
namedt_data = conn.raw_sql("""
        SELECT permno, namedt
        FROM crsp.stocknames
        WHERE namedt IS NOT NULL
    """)

In [123]:
merged_data = pd.merge(compustat_data, link_table, on='gvkey', how='left')
merged_data = pd.merge(merged_data, namedt_data, on='permno', how='left')
# Identify the earliest linkdt for each firm
earliest_linkdt = merged_data.groupby('gvkey')['linkdt'].min().reset_index()
earliest_linkdt.rename(columns={'linkdt': 'earliest_linkdt'}, inplace=True)


In [124]:
merged_data.to_csv('table7_original.csv', index=False)

In [125]:
merged_data.to_pickle('table7_original.pkl')

In [78]:
merged_data = pd.read_csv('table7_original.csv')

In [224]:
merged_data = pd.read_pickle('table7_original.pkl')

In [225]:
merged_data = pd.merge(merged_data, earliest_linkdt, on='gvkey', how='left')

merged_data = merged_data.sort_values(by=['gvkey', 'fyear', 'linktype'], ascending=[True, True, False])

merged_data = merged_data.drop_duplicates(subset=['gvkey', 'fyear'])

merged_data['xrd'] = merged_data['xrd'].fillna(0)

merged_data['xad'] = merged_data['xad'].fillna(0)

merged_data = merged_data[merged_data['exchg'] == 11]

In [226]:
merged_data['firm_age'] = merged_data['fyear'] - pd.to_datetime(merged_data['namedt']).dt.year

In [227]:
merged_data = merged_data[merged_data['firm_age'] >= 0]

In [228]:
merged_data['mve'] = merged_data['csho'] * merged_data['prcc_f']
merged_data['market_value_of_assets'] = merged_data['at'] - merged_data['ceq'] + merged_data['mve']
merged_data['market_debt_ratio'] = (merged_data['dlc'] + merged_data['dltt']) / (merged_data['at'] - merged_data['ceq'] + merged_data['mve'])

merged_data['ln_market_value_of_assets'] = np.log(merged_data['market_value_of_assets'])
merged_data['ln_firm_age'] = np.log(1 + merged_data['firm_age'])

merged_data['profits_to_sales'] = merged_data['oibdp'] / merged_data['sale']
merged_data['tangible_assets'] = merged_data['ppent'] / merged_data['at']
merged_data['market_to_book_assets'] = merged_data['market_value_of_assets'] / merged_data['at']
merged_data['advertising_to_sales'] = merged_data['xad'] / merged_data['sale']
merged_data['rd_to_sales'] = merged_data['xrd'] / merged_data['sale']
merged_data['rd_positive'] = (merged_data['xrd'] > 0).astype(int)


In [229]:
merged_data = merged_data.sort_values(by=['gvkey', 'fyear'])
merged_data['prev_year_dvc'] = merged_data.groupby('gvkey')['dvc'].shift(1)

In [230]:
# merged_data = merged_data.sort_values(by=['gvkey', 'fyear'])
independent_vars = ['ln_market_value_of_assets', 'ln_firm_age', 'profits_to_sales', 'tangible_assets', 'market_to_book_assets', 'advertising_to_sales', 'rd_to_sales', 'rd_positive']
for var in independent_vars:
    merged_data[var] = merged_data.groupby('gvkey')[var].shift(1)

In [231]:
merged_data = merged_data[merged_data['prev_year_dvc'] > 0]

In [232]:
merged_data = merged_data.dropna(subset=[f'{var}' for var in independent_vars])

In [233]:
merged_data = merged_data.dropna(subset=['market_debt_ratio'])

In [234]:
variables_to_winsorize = ['profits_to_sales', 'tangible_assets', 'market_to_book_assets', 'advertising_to_sales', 'rd_to_sales']
for var in variables_to_winsorize:
    lower_bound = merged_data[var].quantile(0.01)
    upper_bound = merged_data[var].quantile(0.99)
    merged_data.loc[:, var] = np.clip(merged_data[var], lower_bound, upper_bound)

In [235]:
variables = ['market_debt_ratio', 'ln_market_value_of_assets', 'ln_firm_age', 'profits_to_sales', 'tangible_assets', 'market_to_book_assets', 'advertising_to_sales', 'rd_to_sales', 'rd_positive']
summary_stats = merged_data[variables].describe().transpose()
summary_stats['count'] = summary_stats['count'].astype(int)
summary_stats = summary_stats[['count', 'mean', 'std', 'min', '50%', 'max']]
summary_stats.columns = ['Number of Observations', 'Mean', 'Standard Deviation', 'Minimum', 'Median', 'Maximum']

In [236]:
print(summary_stats)

                           Number of Observations      Mean  \
market_debt_ratio                           39668  0.219739   
ln_market_value_of_assets                   39668  6.821959   
ln_firm_age                                 39668  2.737411   
profits_to_sales                            39668  0.195611   
tangible_assets                             39668  0.377306   
market_to_book_assets                       39668  1.460845   
advertising_to_sales                        39668  0.008161   
rd_to_sales                                 39668  0.009116   
rd_positive                                 39668  0.348593   

                           Standard Deviation   Minimum    Median    Maximum  
market_debt_ratio                    0.164205  0.000000  0.193632   0.978670  
ln_market_value_of_assets            1.923990  1.006862  6.723540  13.677366  
ln_firm_age                          0.990250  0.000000  2.890372   4.356709  
profits_to_sales                     0.148980  0.0027

In [95]:
merged_data['original_fyear'] = merged_data['fyear']
year_dummies = pd.get_dummies(merged_data['fyear'], prefix='fyear', drop_first=True)
merged_data = pd.concat([merged_data, year_dummies], axis=1)
merged_data['fyear'] = merged_data['original_fyear']

In [237]:
formula_with_year_dummies = 'market_debt_ratio ~ ' + ' + '.join(independent_vars) + ' + C(fyear)'

In [238]:
formula_fm = 'market_debt_ratio ~ ' + ' + '.join(independent_vars)

In [155]:
merged_data = merged_data.sort_values(by=['gvkey', 'fyear'])

In [239]:
from linearmodels.panel import PanelOLS
from linearmodels.panel import FamaMacBeth

In [240]:
# merged_data.reset_index(inplace=True)
model_white = smf.ols(formula=formula_with_year_dummies, data=merged_data).fit(cov_type='HC0')
model_firm = smf.ols(formula=formula_with_year_dummies, data=merged_data).fit(cov_type='cluster', cov_kwds={'groups': merged_data['gvkey']})
model_year = smf.ols(formula=formula_with_year_dummies, data=merged_data).fit(cov_type='cluster', cov_kwds={'groups': merged_data['fyr']})
model_firm_year = smf.ols(formula=formula_with_year_dummies, data=merged_data).fit(cov_type='cluster', cov_kwds={'groups': merged_data[['gvkey', 'fyr']].apply(tuple, axis=1)})

# Perform the Fama-MacBeth regression analysis
merged_data.set_index(['gvkey', 'fyear'], inplace=True)
model_fm = FamaMacBeth.from_formula(formula_fm, data=merged_data)
results_fm = model_fm.fit()

# Function to format coefficients and standard errors with significance stars
def format_coefficients(model, coef_name, is_fm=False):
    coef = model.params[coef_name]
    se = model.std_errors[coef_name] if is_fm else model.bse[coef_name]
    pval = model.pvalues[coef_name]
    stars = ''
    if pval < 0.01:
        stars = '**'
    elif pval < 0.05:
        stars = '*'
    return f'{coef:.4f}{stars}\n({se:.4f})'

# Create a DataFrame to store the results
results_table = pd.DataFrame(index=independent_vars, columns=['I', 'II', 'III', 'IV', 'V'])

# Fill the DataFrame with formatted coefficients and standard errors
for var in independent_vars:
    results_table.loc[var, 'I'] = format_coefficients(model_white, f'{var}')
    results_table.loc[var, 'II'] = format_coefficients(model_firm, f'{var}')
    results_table.loc[var, 'III'] = format_coefficients(model_year, f'{var}')
    results_table.loc[var, 'IV'] = format_coefficients(model_firm_year, f'{var}')
    results_table.loc[var, 'V'] = format_coefficients(results_fm, f'{var}', is_fm=True)

# Add R-squared and method/standard error type information
results_table.loc['R-squared'] = [
    f'{model_white.rsquared:.4f}',
    f'{model_firm.rsquared:.4f}',
    f'{model_year.rsquared:.4f}',
    f'{model_firm_year.rsquared:.4f}',
    f'{results_fm.rsquared:.4f}'
]
results_table.loc['Coefficient estimates'] = ['OLS', 'OLS', 'OLS', 'OLS', 'FM']
results_table.loc['Standard errors'] = ['White', 'CL - F', 'CL - T', 'CL - F&T', 'FM']

# Print the results table
print(results_table)

                                             I                   II  \
ln_market_value_of_assets     0.0004\n(0.0004)     0.0004\n(0.0014)   
ln_firm_age                   0.0006\n(0.0008)     0.0006\n(0.0022)   
profits_to_sales            0.1725**\n(0.0083)   0.1725**\n(0.0253)   
tangible_assets             0.1616**\n(0.0033)   0.1616**\n(0.0109)   
market_to_book_assets      -0.0776**\n(0.0009)  -0.0776**\n(0.0024)   
advertising_to_sales         -0.0560\n(0.0315)    -0.0560\n(0.0879)   
rd_to_sales                -0.1566**\n(0.0380)    -0.1566\n(0.0938)   
rd_positive                -0.0172**\n(0.0018)  -0.0172**\n(0.0051)   
R-squared                               0.3090               0.3090   
Coefficient estimates                      OLS                  OLS   
Standard errors                          White               CL - F   

                                           III                   IV  \
ln_market_value_of_assets     0.0004\n(0.0020)     0.0004\n(0.0014)   
ln_fi

In [174]:
model_white.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      market_debt_ratio   R-squared:                       0.213
Model:                            OLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                     228.5
Date:                Wed, 26 Feb 2025   Prob (F-statistic):               0.00
Time:                        21:02:15   Log-Likelihood:                 19643.
No. Observations:               38578   AIC:                        -3.919e+04
Df Residuals:                   38532   BIC:                        -3.880e+04
Df Model:                          45                                         
Covariance Type:                  HC0                                         
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                     0.1394      0.009     15.993      0.000       0.122       0.156
C(fyear)[T.1967]             -0.0198      0.009     -2.267      0.023      -0.037      -0.003
C(fyear)[T.1968]             -0.0211      0.008     -2.512      0.012      -0.038      -0.005
C(fyear)[T.1969]              0.0287      0.009      3.271      0.001       0.011       0.046
C(fyear)[T.1970]              0.0483      0.009      5.386      0.000       0.031       0.066
C(fyear)[T.1971]              0.0353      0.009      3.987      0.000       0.018       0.053
C(fyear)[T.1972]              0.0348      0.009      3.992      0.000       0.018       0.052
C(fyear)[T.1973]              0.0796      0.009      8.965      0.000       0.062       0.097
C(fyear)[T.1974]              0.1201      0.009     13.314      0.000       0.102       0.138
C(fyear)[T.1975]              0.0785      0.009      8.779      0.000       0.061       0.096
C(fyear)[T.1976]              0.0457      0.009      5.340      0.000       0.029       0.062
C(fyear)[T.1977]              0.0477      0.008      5.647      0.000       0.031       0.064
C(fyear)[T.1978]              0.0556      0.009      6.392      0.000       0.039       0.073
C(fyear)[T.1979]              0.0489      0.008      5.781      0.000       0.032       0.066
C(fyear)[T.1980]              0.0352      0.008      4.183      0.000       0.019       0.052
C(fyear)[T.1981]              0.0427      0.008      5.102      0.000       0.026       0.059
C(fyear)[T.1982]              0.0332      0.008      3.978      0.000       0.017       0.050
C(fyear)[T.1983]              0.0020      0.008      0.238      0.812      -0.014       0.018
C(fyear)[T.1984]              0.0095      0.008      1.164      0.244      -0.007       0.026
C(fyear)[T.1985]              0.0025      0.008      0.303      0.762      -0.014       0.019
C(fyear)[T.1986]              0.0058      0.008      0.706      0.480      -0.010       0.022
C(fyear)[T.1987]              0.0180      0.009      2.115      0.034       0.001       0.035
C(fyear)[T.1988]              0.0214      0.009      2.455      0.014       0.004       0.038
C(fyear)[T.1989]              0.0206      0.009      2.349      0.019       0.003       0.038
C(fyear)[T.1990]              0.0380      0.009      4.290      0.000       0.021       0.055
C(fyear)[T.1991]              0.0173      0.009      1.961      0.050    1.08e-05       0.035
C(fyear)[T.1992]              0.0069      0.009      0.803      0.422      -0.010       0.024
C(fyear)[T.1993]             -0.0089      0.008     -1.052      0.293      -0.026       0.008
C(fyear)[T.1994]             -0.0021      0.008     -0.249      0.803      -0.019       0.014
C(fyear)[T.1995]             -0.0165      0.008     -1.991      0.046      

In [209]:
# Part E. replication of Table 2 of Gormley and Matsa (2014)
query = """
SELECT a.gvkey,
       a.datadate,
       a.fyear,
       b.sic,
       a.dltt,
       a.dlc,
       a.at,
       a.ppent,
       a.sale,
       a.ni,
       a.act,
       a.lct,
       a.re,
       a.oiadp,
       a.prcc_f,
       a.csho,
       a.lt,
       a.xint,
       a.txt
FROM comp.funda a
LEFT JOIN comp.company b
       ON a.gvkey = b.gvkey
WHERE a.indfmt = 'INDL'
  AND a.datafmt = 'STD'
  AND a.popsrc = 'D'
  AND a.consol = 'C'
  AND a.fyear BETWEEN 1950 AND 2010
"""
df = conn.raw_sql(query)

In [210]:
df.to_pickle('table2_original.pkl')

In [182]:
conn.describe_table(library='comp', table='funda')

Approximately 910351 rows in comp.funda.


,name,nullable,type,comment
0,gvkey,True,VARCHAR(6),Global Company Key
1,datadate,True,DATE,Data Date
2,fyear,True,INTEGER,Data Year - Fiscal
3,indfmt,True,VARCHAR(12),Industry Format
4,consol,True,VARCHAR(2),Level of Consolidation - Company Annual Descri...
...,...,...,...,...
943,au,True,VARCHAR(8),Auditor
944,auop,True,VARCHAR(8),Auditor Opinion
945,auopic,True,VARCHAR(1),Auditor Opinion - Internal Control
946,ceoso,True,VARCHAR(1),Chief Executive Officer SOX Certification


In [186]:
conn.describe_table(library='comp', table='company')

Approximately 54410 rows in comp.company.


,name,nullable,type,comment
0,conm,True,VARCHAR(70),Company Name
1,gvkey,True,VARCHAR(6),Global Company Key - Company
2,add1,True,VARCHAR(65),Address Line 1
3,add2,True,VARCHAR(65),Address Line 2
4,add3,True,VARCHAR(65),Address Line 3
5,add4,True,VARCHAR(65),Address Line 4
6,addzip,True,VARCHAR(20),Postal Code
7,busdesc,True,VARCHAR(2000),S&P Business Description
8,cik,True,VARCHAR(10),CIK Number
9,city,True,VARCHAR(100),City


In [211]:
df['datadate'] = pd.to_datetime(df['datadate'])

In [212]:
df['sic'] = pd.to_numeric(df['sic'], errors='coerce')
df = df[~df['sic'].between(6000, 6999)]
df = df[~df['sic'].between(4900, 4999)]

In [213]:
# Total Debt / Total Assets
df['debt_assets'] = (df['dlc'] + df['dltt']) / df['at']

# Fixed Assets / Total Assets
df['fixed_assets'] = df['ppent'] / df['at']

# Ln(Sales)
df['ln_sales'] = np.log(df['sale'])

# Return on Assets
df['roa'] = df['ni'] / df['at']

# Modified Altman Z-score
df['wc'] = df['act'] - df['lct']
df['wc_ta'] = df['wc'] / df['at']
df['re_ta'] = df['re'] / df['at']
df['ebit'] = df['oiadp'].fillna(df['ni'] + df['xint'] + df['txt'])
df['ebit_ta'] = df['ebit'] / df['at']
df['mve'] = df['prcc_f'] * df['csho']
df['mve_tl'] = df['mve'] / df['lt'].replace(0, np.nan)
df['sales_ta'] = df['sale'] / df['at']
df['z_score'] = 1.2*df['wc_ta'] + 1.4*df['re_ta'] + 3.3*df['ebit_ta'] + 0.6*df['mve_tl'] + 1.0*df['sales_ta']

# Market-to-Book Ratio
df['market_to_book'] = (df['mve'] + df['lt']) / df['at']

# Drop missing values
df = df.dropna(subset=['debt_assets', 'fixed_assets', 'ln_sales', 'roa', 'z_score', 'market_to_book'])

/Users/friend/anaconda3/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/friend/anaconda3/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [214]:
df['modified_z'] = (
    1.2 * (df['wc'] / df['at']) +
    1.4 * (df['re'] / df['at']) +
    3.3 * (df['oiadp'] / df['at']) +
    0.6 * (df['mve'] / df['lt']) +
    0.999 * (df['sale'] / df['at'])
)

In [215]:
def winsorize(series, lower=0.01, upper=0.99):
    return np.clip(series, series.quantile(lower), series.quantile(upper))

df['fa_ta'] = winsorize(df['fixed_assets'] / df['at'])
df['ln_sales'] = winsorize(np.log(df['sale']))
df['roa'] = winsorize(df['roa'])
df['zscore'] = winsorize(df['modified_z'])
df['mtb'] = winsorize(df['market_to_book'])
df['leverage'] = winsorize(df['debt_assets'])

/Users/friend/anaconda3/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/friend/anaconda3/lib/python3.10/site-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


In [216]:
all_var = ['fa_ta', 'ln_sales', 'roa', 'zscore', 'mtb', 'leverage']
df = df.dropna(subset=all_var)

In [217]:
df = df[df['ln_sales'] > -100]


In [218]:
firm_year_counts = df.groupby('gvkey').size()
firms_with_multiple_years = firm_year_counts[firm_year_counts > 1].index
df = df[df['gvkey'].isin(firms_with_multiple_years)].copy()

In [ ]:
df.reset_index(inplace=True)

In [219]:
# Define the regression formula
formula = 'leverage ~ fa_ta + ln_sales + roa + zscore + mtb'

# (1) OLS
model_ols = smf.ols(formula=formula, data=df).fit(cov_type='cluster', cov_kwds={'groups': df['gvkey']})

# (2) AdjY: Subtract the firm’s average Y from each observation (excluding the current one)
df['firm_avg_leverage'] = df.groupby('gvkey')['leverage'].transform(lambda x: (x.sum() - x) / (len(x) - 1))
df['adjy_leverage'] = df['leverage'] - df['firm_avg_leverage']
model_adjy = smf.ols(formula='adjy_leverage ~ fa_ta + ln_sales + roa + zscore + mtb', data=df).fit(cov_type='cluster', cov_kwds={'groups': df['gvkey']})

# (3) AvgE: Subtract the firm’s average residual from each observation’s Y
ols_for_resid = smf.ols(formula=formula, data=df).fit()
df['resid'] = ols_for_resid.resid
df['firm_avg_resid'] = df.groupby('gvkey')['resid'].transform(lambda x: (x.sum() - x) / (len(x) - 1))
df['avge_leverage'] = df['leverage'] - df['firm_avg_resid']
model_avge = smf.ols(formula='avge_leverage ~ fa_ta + ln_sales + roa + zscore + mtb', data=df).fit(cov_type='cluster', cov_kwds={'groups': df['gvkey']})

# (4) FE: Fixed-effects model
df.set_index(['gvkey', 'fyear'], inplace=True)
model_fe = PanelOLS.from_formula(formula + ' + EntityEffects', data=df).fit(cov_type='clustered', cluster_entity=True)

In [202]:
model_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               leverage   R-squared:                       0.270
Model:                            OLS   Adj. R-squared:                  0.270
Method:                 Least Squares   F-statistic:                     1143.
Date:                Wed, 26 Feb 2025   Prob (F-statistic):               0.00
Time:                        23:26:20   Log-Likelihood:                 9008.4
No. Observations:              209742   AIC:                        -1.800e+04
Df Residuals:                  209736   BIC:                        -1.794e+04
Df Model:                           5                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2193      0.004     60.028      0.000       0.212       0.226
fa_ta          0.2278      0.030      7.476      0.000       0.168       0.287
ln_sales       0.0091      0.001     14.752      0.000       0.008       0.010
roa           -0.0493      0.005    -10.704      0.000      -0.058      -0.040
zscore        -0.0090      0.000    -49.552      0.000      -0.009      -0.009
mtb            0.0108      0.001     14.534      0.000       0.009       0.012
==============================================================================
Omnibus:                    55517.910   Durbin-Watson:                   1.967
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           396288.968
Skew:                           1.088   Prob(JB):                         0.00
Kurtosis:                       9.373   Cond. No.                         241.
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [222]:

def format_coefficients(model, coef_name):
    coef = model.params[coef_name]
    if hasattr(model, 'bse'):
        se = model.bse[coef_name]  # Use bse for standard errors in OLS models
    else:
        se = model.std_errors[coef_name]  # Use std_errors for fixed-effects models
    pval = model.pvalues[coef_name]
    stars = ''
    if pval < 0.01:
        stars = '***'
    elif pval < 0.05:
        stars = '**'
    elif pval < 0.1:
        stars = '*'
    return f'{coef:.3f}{stars}\n({se:.3f})'

# Create a DataFrame to store the results
results_table = pd.DataFrame(index=['Fixed Assets/Total Assets', 'Ln(Sales)', 'Return on Assets', 'Z-score', 'Market-to-book Ratio', 'Observations', 'R2'], columns=['OLS', 'AdjY', 'AvgE', 'FE'])

# Fill the DataFrame with formatted coefficients and standard errors
results_table.loc['Fixed Assets/Total Assets', 'OLS'] = format_coefficients(model_ols, 'fa_ta')
results_table.loc['Ln(Sales)', 'OLS'] = format_coefficients(model_ols, 'ln_sales')
results_table.loc['Return on Assets', 'OLS'] = format_coefficients(model_ols, 'roa')
results_table.loc['Z-score', 'OLS'] = format_coefficients(model_ols, 'zscore')
results_table.loc['Market-to-book Ratio', 'OLS'] = format_coefficients(model_ols, 'mtb')
results_table.loc['Observations', 'OLS'] = f'{int(model_ols.nobs)}'
results_table.loc['R2', 'OLS'] = f'{model_ols.rsquared:.2f}'

results_table.loc['Fixed Assets/Total Assets', 'AdjY'] = format_coefficients(model_adjy, 'fa_ta')
results_table.loc['Ln(Sales)', 'AdjY'] = format_coefficients(model_adjy, 'ln_sales')
results_table.loc['Return on Assets', 'AdjY'] = format_coefficients(model_adjy, 'roa')
results_table.loc['Z-score', 'AdjY'] = format_coefficients(model_adjy, 'zscore')
results_table.loc['Market-to-book Ratio', 'AdjY'] = format_coefficients(model_adjy, 'mtb')
results_table.loc['Observations', 'AdjY'] = f'{int(model_adjy.nobs)}'
results_table.loc['R2', 'AdjY'] = f'{model_adjy.rsquared:.2f}'

results_table.loc['Fixed Assets/Total Assets', 'AvgE'] = format_coefficients(model_avge, 'fa_ta')
results_table.loc['Ln(Sales)', 'AvgE'] = format_coefficients(model_avge, 'ln_sales')
results_table.loc['Return on Assets', 'AvgE'] = format_coefficients(model_avge, 'roa')
results_table.loc['Z-score', 'AvgE'] = format_coefficients(model_avge, 'zscore')
results_table.loc['Market-to-book Ratio', 'AvgE'] = format_coefficients(model_avge, 'mtb')
results_table.loc['Observations', 'AvgE'] = f'{int(model_avge.nobs)}'
results_table.loc['R2', 'AvgE'] = f'{model_avge.rsquared:.2f}'

results_table.loc['Fixed Assets/Total Assets', 'FE'] = format_coefficients(model_fe, 'fa_ta')
results_table.loc['Ln(Sales)', 'FE'] = format_coefficients(model_fe, 'ln_sales')
results_table.loc['Return on Assets', 'FE'] = format_coefficients(model_fe, 'roa')
results_table.loc['Z-score', 'FE'] = format_coefficients(model_fe, 'zscore')
results_table.loc['Market-to-book Ratio', 'FE'] = format_coefficients(model_fe, 'mtb')
results_table.loc['Observations', 'FE'] = f'{int(model_fe.nobs)}'
results_table.loc['R2', 'FE'] = f'{model_fe.rsquared:.2f}'

# Print the results table
print(results_table)

                                          OLS                AdjY  \
Fixed Assets/Total Assets   0.264***\n(0.037)   0.119***\n(0.025)   
Ln(Sales)                   0.009***\n(0.001)   0.009***\n(0.000)   
Return on Assets           -0.047***\n(0.005)  -0.034***\n(0.004)   
Z-score                    -0.010***\n(0.000)  -0.005***\n(0.000)   
Market-to-book Ratio        0.013***\n(0.001)   0.005***\n(0.001)   
Observations                           208262              208262   
R2                                       0.30                0.13   

                                         AvgE                  FE  
Fixed Assets/Total Assets   0.204***\n(0.022)   0.137***\n(0.043)  
Ln(Sales)                   0.009***\n(0.000)   0.008***\n(0.001)  
Return on Assets           -0.063***\n(0.004)  -0.071***\n(0.005)  
Z-score                    -0.009***\n(0.000)  -0.008***\n(0.000)  
Market-to-book Ratio        0.015***\n(0.000)   0.015***\n(0.001)  
Observations                           

In [223]:
results_table.to_csv('table5.csv')